In [ ]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [ ]:
aoai_embedding_deployment_name=""
aoai_chat_deployment_name=""

In [ ]:
azdo_org_uri=""
azdo_pat=""
azdo_project=""

In [ ]:
from AzdoBacklogLoader import AzdoBacklogLoader
from AzdoPullRequestLoader import AzdoPullRequestLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

backlog_loader = AzdoBacklogLoader(azdo_org_uri, azdo_pat, azdo_project)
pr_loader = AzdoPullRequestLoader(azdo_org_uri, azdo_pat, azdo_project)
raw_docs = backlog_loader.load() + pr_loader.load()

chunked_docs = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=100).split_documents(raw_docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment=aoai_embedding_deployment_name)
db = FAISS.from_documents(chunked_docs, embeddings)

retriever = db.as_retriever()

In [ ]:
from langchain.chat_models.azure_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = AzureChatOpenAI(azure_deployment=aoai_chat_deployment_name)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("What was the most recent completed work item?")